<a href="https://colab.research.google.com/github/HareeshwarKarthikeyan/Visual-BMI-Estimation/blob/main/Face_embeddings_using_VGG_FACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the necessary modules

In [ ]:
import numpy as np
import pandas as pd
import face_recognition
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

In [ ]:
p install face_recognition

     |████████████████████████████████| 100.2MB 41kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=13dfbde5323db7995d7a00aaf96377a87740e61e038d30b345ddc921bf9338c0
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


Importing the BMI data from the csv file

In [ ]:
image_csv=pd.read_csv("/gdrive/My Drive/data/annotation.csv")
image_csv.head

<bound method NDFrame.head of       image  height  weight        BMI
0     f_001    1.55    61.0  25.390219
1     f_002    1.76    85.0  27.440599
2     f_003    1.78    56.0  17.674536
3     f_004    1.63    63.0  23.711845
4     f_005    1.76    54.0  17.432851
...     ...     ...     ...        ...
1021  m_509    1.91   116.0  31.797374
1022  m_510    1.93   111.0  29.799458
1023  m_511    1.88   109.0  30.839746
1024  m_512    1.78    75.0  23.671254
1025  m_513    2.21   137.0  28.050204

[1026 rows x 4 columns]>

Finding out the number of images in the data folder

In [ ]:
data_folder = "/gdrive/My Drive/data/data"
from glob import glob
all_files = glob(data_folder+"/*")
all_jpgs = sorted([img for img in all_files if ".jpg" in img or ".jpeg" in img or "JPG" in img])
print("Total {} photos ".format(len(all_jpgs)))

Total 1026 photos 


Adding the image path column to the dataframe 

In [ ]:
from pathlib import Path as p
image_csv['image']=all_jpgs

In [ ]:
import sys
import os
import bz2
filepath ="/gdrive/My Drive/data/mmod_human_face_detector.dat.bz2"
zipfile = bz2.BZ2File(filepath) # open the file
data = zipfile.read() # get the decompressed data
newfilepath = filepath[:-4] # assuming the filepath ends with .bz2
open(newfilepath, 'wb').write(data) # write a uncompressed file

Define VGG_FACE_MODEL architecture


In [107]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))	
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
# Load VGG Face model weights
model.load_weights('/gdrive/My Drive/data/vgg_face_weights.h5')

We need not classify whether the image consists of a face or not so we remove the final softmax layer in the output in order to find out only the embeddings

In [ ]:
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output) 

For every single image we load it with size(224,224) and we find the embedding vector by passing it to the neural network. 

In [ ]:
def get_encoding(image_path):
    picture=load_img(image_path,target_size=(224,224),color_mode='rgb')
    picture=img_to_array(picture)
    img_encode=[]
    print(image_path)
    picture=np.expand_dims(picture,axis=0)
    picture=preprocess_input(picture)
    img_encode=vgg_face(picture)
    return img_encode

In [ ]:
encodings=[]
for images in image_csv.image:
    face_enc = get_encoding(images)
    encodings.append(np.squeeze(K.eval(face_enc)).tolist())

/gdrive/My Drive/data/data/f_001.jpg
/gdrive/My Drive/data/data/f_002.jpg
/gdrive/My Drive/data/data/f_003.jpg
/gdrive/My Drive/data/data/f_004.jpg
/gdrive/My Drive/data/data/f_005.jpg
/gdrive/My Drive/data/data/f_006.jpg
/gdrive/My Drive/data/data/f_007.jpg
/gdrive/My Drive/data/data/f_008.jpg
/gdrive/My Drive/data/data/f_009.jpg
/gdrive/My Drive/data/data/f_010.jpg
/gdrive/My Drive/data/data/f_011.jpg
/gdrive/My Drive/data/data/f_012.jpg
/gdrive/My Drive/data/data/f_013.jpg
/gdrive/My Drive/data/data/f_014.jpg
/gdrive/My Drive/data/data/f_015.jpg
/gdrive/My Drive/data/data/f_016.jpg
/gdrive/My Drive/data/data/f_017.jpg
/gdrive/My Drive/data/data/f_018.jpg
/gdrive/My Drive/data/data/f_019.jpg
/gdrive/My Drive/data/data/f_020.jpg
/gdrive/My Drive/data/data/f_021.jpg
/gdrive/My Drive/data/data/f_022.jpg
/gdrive/My Drive/data/data/f_023.jpg
/gdrive/My Drive/data/data/f_024.jpg
/gdrive/My Drive/data/data/f_025.jpg
/gdrive/My Drive/data/data/f_026.jpg
/gdrive/My Drive/data/data/f_027.jpg
/

In [ ]:
x=np.array(encodings)
print(x.shape)

(1026, 2622)


In [103]:
y_BMI = image_csv.BMI.values

x-contains embeddings for 1026 images

y_BMI-contains the corresponding BMI values for the 1026 images

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_BMI_train, y_BMI_test = train_test_split(x,y_BMI, random_state=1)

In [105]:
from sklearn.kernel_ridge import KernelRidge
from sklearn import  linear_model
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
model_BMI = linear_model.LinearRegression()
model_BMI = model_BMI.fit(X_train,np.log(y_BMI_train))
report_goodness(model_BMI,X_test,y_BMI_test)

Mean squared error: 0.03
Variance score: -0.23
Model Performance
Average Error: 0.1402 degrees.
Accuracy = 95.55%.
